In [1]:
import teehr
from pathlib import Path
import shutil
import xarray as xr
import geopandas as gpd
from teehr.models.pydantic_table_models import (
    Attribute,
    Configuration,
    Variable
)

In [2]:
# Set a path to the directory where the evaluation will be created
TEST_STUDY_DIR = Path(Path().home(), "temp", "test_study")
shutil.rmtree(TEST_STUDY_DIR, ignore_errors=True)
TEST_STUDY_DIR.mkdir(parents=True, exist_ok=True)

# Set a path to the directory where the test data is stored
TEST_DATA_DIR = Path(Path(teehr.__file__).parent.parent.parent, "tests/data/v0_3_test_study")
GEOJSON_GAGES_FILEPATH = Path(TEST_DATA_DIR, "geo", "gages.geojson")
PRIMARY_TIMESERIES_FILEPATH = Path(
    TEST_DATA_DIR, "timeseries", "test_short_obs.parquet"
)
CROSSWALK_FILEPATH = Path(TEST_DATA_DIR, "geo", "crosswalk.csv")
SECONDARY_TIMESERIES_FILEPATH = Path(
    TEST_DATA_DIR, "timeseries", "test_short_fcast.parquet"
)
GEO_FILEPATH = Path(TEST_DATA_DIR, "geo")

Timeseries, location crosswalks, and location attributes can also be loaded from netcdf files.

In [12]:
# Create a new Evaluation object
shutil.rmtree(TEST_STUDY_DIR, ignore_errors=True)
TEST_STUDY_DIR.mkdir(parents=True, exist_ok=True)

ev = teehr.Evaluation(dir_path=TEST_STUDY_DIR)

# Enable logging
ev.enable_logging()

# Clone the template
ev.clone_template()

In [13]:
# The primary locations file with geometry
# (TEEHR currently only supports formats readable by geopandas)
NETCDF_GEO_FILEPATH = Path(
    "/Users/mdenno/repos/teehr/tests/data/test_study/geo/summa_locations.parquet"
)
gpd.read_parquet(NETCDF_GEO_FILEPATH)

,id,name,geometry
0,170300010101,Site A,POINT (-80.71918 35.17611)
1,171003010703,Site B,POINT (-80.71308 35.15839)


In [14]:
# Load the locations and add the configuration and variable domain entries
ev.locations.load_spatial(in_path=NETCDF_GEO_FILEPATH)

ev.configurations.add(
    Configuration(
        name="summa",
        type="primary",
        description="Summa Runoff Data"
    )
)

ev.variables.add(
    Variable(
        name="runoff",
        long_name="runoff"
    )
)

In [15]:
# Here is an example considered a primary timeseries file in netcdf format.
NETCDF_TIMESERIES_FILEPATH = Path(
    "/Users/mdenno/repos/teehr/tests/data/test_study/timeseries/summa.example.nc"
)
# Look at the contents of the netcdf file to determine the field mapping
xr.open_dataset(NETCDF_TIMESERIES_FILEPATH)

<xarray.Dataset> Size: 1MB
Dimensions:                     (time: 11688, hru: 2, gru: 2)
Coordinates:
  * time                        (time) datetime64[ns] 94kB 1991-01-02 ... 202...
  * hru                         (hru) int64 16B 170300010101 171003010703
  * gru                         (gru) int64 16B 170300010101 171003010703
Data variables: (12/13)
    pptrate_mean                (time, hru) float32 94kB ...
    airtemp_mean                (time, hru) float32 94kB ...
    averageRoutedRunoff_mean    (time, gru) float32 94kB ...
    scalarCanopyWat             (time, hru) float32 94kB ...
    scalarSWE                   (time, hru) float32 94kB ...
    scalarAquiferStorage        (time, hru) float32 94kB ...
    ...                          ...
    scalarSurfaceRunoff_mean    (time, hru) float32 94kB ...
    scalarAquiferBaseflow_mean  (time, hru) float32 94kB ...
    scalarTotalET_mean          (time, hru) float32 94kB ...
    scalarTotalRunoff_mean      (time, hru) float32 94kB ...
    hruId                       (hru) int64 16B ...
    gruId                       (gru) int64 16B ...
Attributes: (12/32)
    summaVersion:     undefined
    buildTime:        Sun Jan  8 06:16:21 MST 2023
    gitBranch:        develop-0-g357c1fb
    gitHash:          357c1fb328eb81d38272182a8031ce6e5f4f6947
    soilCatTbl:       STAS
    vegeParTbl:       MODIFIED_IGBP_MODIS_NOAH
    ...               ...
    snowLayers:       CLM_2010
    compaction:       anderson
    thCondSnow:       jrdn1991
    thCondSoil:       funcSoilWet
    spatial_gw:       localColumn
    subRouting:       timeDlay

The TEEHR Timeseries table schema includes fields:

- reference_time
- value_time
- configuration_name
- unit_name
- variable_name
- value
- location_id

In [16]:
# Map the fields in the netcdf file to the fields in the database, defining the constant values
summa_field_mapping = {
    "time": "value_time",
    "averageRoutedRunoff_mean": "value",
    "gru": "location_id"
}
summa_constant_field_values = {
    "unit_name": "m^3/s",
    "variable_name": "runoff",
    "configuration_name": "summa",
    "reference_time": None
}

In [17]:
# Load the timeseries data, mapping over the fields and setting constants
ev.primary_timeseries.load_netcdf(
    in_path=NETCDF_TIMESERIES_FILEPATH,
    field_mapping=summa_field_mapping,
    constant_field_values=summa_constant_field_values
)

In [18]:
ev.primary_timeseries.to_pandas().head()

,reference_time,value_time,value,unit_name,location_id,configuration_name,variable_name
0,NaT,1991-01-02,1.809680e-08,m^3/s,170300010101,summa,runoff
1,NaT,1991-01-02,1.330444e-08,m^3/s,171003010703,summa,runoff
2,NaT,1991-01-03,1.755817e-08,m^3/s,170300010101,summa,runoff
3,NaT,1991-01-03,1.300133e-08,m^3/s,171003010703,summa,runoff
4,NaT,1991-01-04,1.704826e-08,m^3/s,170300010101,summa,runoff
